In [ ]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import numpy as np
from math import pi

from bokeh.layouts import gridplot, row, column, widgetbox
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, BasicTicker, ColorBar, ColumnDataSource, LinearColorMapper, PrintfTickFormatter, Slider
from bokeh.plotting import figure
from bokeh.transform import dodge, transform, cumsum
from bokeh.io import export_png, curdoc
from bokeh.models import Range1d
from bokeh.palettes import Category20c, Inferno256, Category10

In [ ]:
# GTD = pd.read_excel("Datasets/GTD_Dataset.xlsx")
# GTD.shape

In [ ]:
# GTD.head()

In [ ]:
# GTD.to_pickle("Datasets/GTD_Dataset.pkl")

In [22]:
GTD_full = pd.read_pickle("Datasets/GTD_Dataset.pkl")
GTD_full.shape

(191464, 135)

In [6]:
GTD = GTD_full[GTD_full.country_txt != 'United Kingdom']
GTD.shape

(186129, 135)

In [7]:
print(GTD.nkill.sum(),GTD.nwound.sum()) 

431570.0 543616.0


In [8]:
UK = GTD_full[GTD_full.country_txt == 'United Kingdom']
UK.shape

(5335, 135)

In [9]:
print(UK.nkill.sum(),UK.nwound.sum()) 

3411.0 6138.0


In [10]:
# for col in GTD.columns:
#     print(col)

In [11]:
# len(GTD.eventid.unique())

### Yearwise distribution of events

In [12]:
Yearly_events_G = GTD.groupby(["iyear"]).count()['eventid'].reset_index()
Yearly_events_G= Yearly_events_G.rename(columns = {'iyear' : 'Year',
                                                  'eventid' : 'Attacks'})
Yearly_events_G = Yearly_events_G.sort_values(by=['Year'], ascending = True )
Yearly_events_G['Formatted_Date'] = pd.to_datetime(Yearly_events_G.Year, format='%Y')
Yearly_events_G['Attacks'] = np.log(Yearly_events_G['Attacks'])
Yearly_events_G.head()

,Year,Attacks,Formatted_Date
0,1970,6.460,1970-01-01
1,1971,5.966,1971-01-01
2,1972,5.620,1972-01-01
3,1973,5.649,1973-01-01
4,1974,5.935,1974-01-01


In [13]:
Yearly_events_UK = UK.groupby(["iyear"]).count()['eventid'].reset_index()
Yearly_events_UK= Yearly_events_UK.rename(columns = {'iyear' : 'Year',
                                                    'eventid' : 'Attacks_UK'})
Yearly_events_UK = Yearly_events_UK.sort_values(by=['Year'], ascending = True )
Yearly_events_UK['Formatted_Date'] = pd.to_datetime(Yearly_events_UK.Year, format='%Y')
Yearly_events_UK['Attacks_UK'] = np.log(Yearly_events_UK['Attacks_UK'])
Yearly_events_UK.head()

,Year,Attacks_UK,Formatted_Date
0,1970,2.485,1970-01-01
1,1971,4.394,1971-01-01
2,1972,5.677,1972-01-01
3,1973,5.242,1973-01-01
4,1974,5.313,1974-01-01


In [14]:
merged = Yearly_events_G.merge(Yearly_events_UK, left_on = 'Year', right_on = 'Year', how= 'inner')
merged = merged [['Year','Attacks', 'Attacks_UK', 'Formatted_Date_x']]
merged.head()

,Year,Attacks,Attacks_UK,Formatted_Date_x
0,1970,6.460,2.485,1970-01-01
1,1971,5.966,4.394,1971-01-01
2,1972,5.620,5.677,1972-01-01
3,1973,5.649,5.242,1973-01-01
4,1974,5.935,5.313,1974-01-01


In [15]:
src = ColumnDataSource(merged)
# src_G.data.keys()

output_file("/Users/poonamarora/Desktop/Aleph/Pictures/html/Terrorism_over_the_years.html")
p1 = figure(x_axis_type="datetime", title="")
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'Year'
p1.yaxis.axis_label = 'Number of Terrorist Events (Log scaled)'
p1.y_range = Range1d(0,10)

p1.line('Formatted_Date_x', 'Attacks', line_color='red', source = src, legend = 'Global', line_width = 2.0)
p1.line('Formatted_Date_x', 'Attacks_UK', line_color='green', source = src, legend = 'UK', line_width = 2.0)

p1.axis.major_label_text_font_size = "7pt"
p1.legend.label_text_font_size = "9pt"
p1.legend.location = 'top_left'
hover = HoverTool(tooltips = [
    ('Year', '@Year'),
    ('Global Events', '@Attacks'),
    ('United Kingdom', '@Attacks_UK')])
p1.add_tools(hover)
show(p1)

### Regionwise distribution of events

In [ ]:
Region_wise_events = GTD.groupby(["iyear", "region_txt"]).count()['eventid'].reset_index()
Region_wise_events= Region_wise_events.rename(columns = {'iyear' : 'Year',
                                                         'region_txt' :'Region',
                                                          'eventid' : 'Attacks'})
Region_wise_events = Region_wise_events.sort_values(by=['Year'], ascending = True )
Region_wise_events['Formatted_Date'] = pd.to_datetime(Region_wise_events.Year, format='%Y')
Region_wise_events = pd.pivot_table(Region_wise_events, values='Attacks', index=['Year', 'Formatted_Date'],
                    columns=['Region'], aggfunc=np.sum).reset_index()
Region_wise_events = Region_wise_events.fillna(0)
Region_wise_events.head()

In [ ]:
Region_wise_events.columns

In [ ]:
from bokeh.palettes import Inferno10, Spectral6
src_R = ColumnDataSource(Region_wise_events)

output_file("/Users/poonamarora/Desktop/Aleph/Pictures/html/Regionwise.html")
p2 = figure(x_axis_type="datetime", title="Terrorism activities over the years(1970-2018) in different regions")
p2.grid.grid_line_alpha=0.3
p2.xaxis.axis_label = 'Year'
p2.yaxis.axis_label = 'Number of Terrorist Events'


p2.line('Formatted_Date', 'Australasia & Oceania', line_color='#081d58', source = src_R, legend = '1.Australasia & Oceania')
p2.line('Formatted_Date', 'Central America & Caribbean', line_color='#253494', source = src_R, legend = '2.Central America & Caribbean')
p2.line('Formatted_Date', 'Central Asia', line_color='#225ea8', source = src_R, legend = '3.Central Asia')
p2.line('Formatted_Date', 'East Asia', line_color='#1d91c0', source = src_R, legend = '4.East Asia')
p2.line('Formatted_Date', 'Eastern Europe', line_color='#41b6c4', source = src_R, legend = '5.Eastern Europe')
p2.line('Formatted_Date', 'Middle East & North Africa', line_color='red', source = src_R, legend = '6.Middle East & North Africa', line_width = 2.0)
p2.line('Formatted_Date', 'North America', line_color='#c7e9b4', source = src_R, legend = '7.North America')
p2.line('Formatted_Date', 'South America', line_color='#edf8b1', source = src_R, legend = '8.South America')
p2.line('Formatted_Date', 'South Asia', line_color='green', source = src_R, legend = '9.South Asia', line_width = 2.0)
p2.line('Formatted_Date', 'Southeast Asia', line_color='#CD4247', source = src_R, legend = '10.Southeast Asia')
p2.line('Formatted_Date', 'Sub-Saharan Africa', line_color='blue', source = src_R, legend = '11.Sub-Saharan Africa', line_width = 2.0)
p2.line('Formatted_Date', 'Western Europe', line_color='#000003', source = src_R, legend = '12.Western Europe')

p2.legend.label_text_font_size = "9pt"
p2.legend.click_policy='hide'
p2.legend.location = "top_left"
p2.axis.major_label_text_font_size = "7pt"
hover = HoverTool(tooltips = [('Year', '@Year')])
p2.add_tools(hover)

show(p2)

### Total events and fatalities in region

In [24]:
Casualities = GTD.groupby("region_txt").agg({'eventid' :'count', 'nkill' : 'sum', 'nwound' : 'sum'}).reset_index()
Casualities = Casualities.rename(columns = {'region_txt' : 'Region',
                                            'eventid' : 'Attacks',
                                            'nkill' : 'Fatalities',
                                            'nwound' : 'Wounded'})
Casualities['Attacks_Percent'] = Casualities['Attacks']*100/Casualities['Attacks'].sum()
Casualities['Fatalities_Percent'] = Casualities['Fatalities']*100/Casualities['Fatalities'].sum()
Casualities['Wounded_Percent'] = Casualities['Wounded']*100/Casualities['Wounded'].sum()
Casualities


,Region,Attacks,Fatalities,Wounded,Attacks_Percent,Fatalities_Percent,Wounded_Percent
0,Australasia & Oceania,304,152.000,263.000,0.163,0.035,0.048
1,Central America & Caribbean,10359,28717.000,9013.000,5.565,6.654,1.658
2,Central Asia,571,1035.000,2019.000,0.307,0.240,0.371
3,East Asia,808,1154.000,9215.000,0.434,0.267,1.695
4,Eastern Europe,5217,7457.000,12097.000,2.803,1.728,2.225
5,Middle East & North Africa,53110,142214.000,219609.000,28.534,32.953,40.398
6,North America,3579,5017.000,27042.000,1.923,1.162,4.974
7,South America,19292,29052.000,17060.000,10.365,6.732,3.138
8,South Asia,48266,112263.000,151199.000,25.931,26.013,27.814
9,Southeast Asia,13363,16257.000,27164.000,7.179,3.767,4.997


In [25]:
  Casualities.sort_values(by=['Attacks_Percent', 'Fatalities_Percent', 'Wounded_Percent'], ascending=False)


,Region,Attacks,Fatalities,Wounded,Attacks_Percent,Fatalities_Percent,Wounded_Percent
5,Middle East & North Africa,53110,142214.000,219609.000,28.534,32.953,40.398
8,South Asia,48266,112263.000,151199.000,25.931,26.013,27.814
10,Sub-Saharan Africa,19752,85020.000,56630.000,10.612,19.700,10.417
7,South America,19292,29052.000,17060.000,10.365,6.732,3.138
9,Southeast Asia,13363,16257.000,27164.000,7.179,3.767,4.997
11,Western Europe,11508,3232.000,12305.000,6.183,0.749,2.264
1,Central America & Caribbean,10359,28717.000,9013.000,5.565,6.654,1.658
4,Eastern Europe,5217,7457.000,12097.000,2.803,1.728,2.225
6,North America,3579,5017.000,27042.000,1.923,1.162,4.974
3,East Asia,808,1154.000,9215.000,0.434,0.267,1.695


In [ ]:
source = ColumnDataSource(data=Casualities)
Region = source.data['Region'].tolist()

output_file("/Users/poonamarora/Desktop/Aleph/Pictures/html/Fatalities_And_Wounded.html")

p3 = figure(x_range=Region, y_range=(0, 45), plot_height=350, title="Total incidents,Casualties and Injured %age in different Regions (1970-2018)")

p3.vbar(x=dodge('Region', -0.25, range=p3.x_range), top='Attacks_Percent', width=0.2, source=source,
       color="darkturquoise", legend="Total Incidents")

p3.vbar(x=dodge('Region',  0.01,  range=p3.x_range), top='Fatalities_Percent', width=0.2, source=source,
       color="indianred", legend="Casualities")

p3.vbar(x=dodge('Region',  0.27,  range=p3.x_range), top='Wounded_Percent', width=0.2, source=source,
       color="forestgreen", legend="Injured")

# p3.xaxis.axis_label = 'Region'
p3.yaxis.axis_label = 'Percentage'

p3.x_range.range_padding = 0.1
p3.xgrid.grid_line_color = None
p3.legend.location = "top_left"
p3.legend.orientation = "vertical"
p3.xaxis.major_label_orientation = math.pi/4
p3.axis.major_label_text_font_size = "7pt"
p3.legend.label_text_font_size = "9pt"

hover = HoverTool(tooltips = [('Terrorist Incidents', '@Attacks_Percent%'),
                             ('Casualties', '@Fatalities_Percent'),
                             ('Injured', '@Wounded_Percent')])
p3.add_tools(hover)
show(p3)

### Explore more terror attack prone regions

In [ ]:
MENA = GTD[GTD['region_txt'] == 'Middle East & North Africa']
MENA.shape

In [ ]:
len(MENA['country_txt'].unique())

In [ ]:
MENA.groupby('country_txt').count()['eventid']

In [ ]:
MN_country = MENA.groupby(['country_txt']).agg({'eventid' :'count', 'nkill' : 'sum', 'nwound' : 'sum'}).reset_index()
MN_country= MN_country.rename(columns = {'country_txt' : 'Country',
                                        'eventid' : 'Attacks',
                                        'nkill' : 'Fatalities',
                                        'nwound' : 'Wounded'})
MN_country['MN_Attacks_Percent'] = MN_country['Attacks']*100/MN_country['Attacks'].sum()
MN_country['MN_Fatalities_Percent'] = MN_country['Fatalities']*100/MN_country['Fatalities'].sum()
MN_country['MN_Wounded_Percent'] = MN_country['Wounded']*100/MN_country['Wounded'].sum()

MN_country = MN_country[MN_country.MN_Attacks_Percent>3.0]
MN_country

In [ ]:
source_MN = ColumnDataSource(data=MN_country)
MN_Country = source_MN.data['Country'].tolist()
# MN_Country

p4 = figure(x_range=MN_Country, y_range=(0, 65), plot_height=350, title="Terrorist events, Casualties and Injuries in Middle East and North Africa")

output_file("/Users/poonamarora/Desktop/Aleph/Pictures/html/Middle_East_and_North_Africa over the years.html")

p4.vbar(x=dodge('Country', -0.25, range=p4.x_range), top='MN_Attacks_Percent', width=0.2, source=source_MN,
       color="darkturquoise", legend="Total Incidents")

p4.vbar(x=dodge('Country',  0.01,  range=p4.x_range), top='MN_Fatalities_Percent', width=0.2, source=source_MN,
       color="indianred", legend="Casualities")

p4.vbar(x=dodge('Country',  0.27,  range=p4.x_range), top='MN_Wounded_Percent', width=0.2, source=source_MN,
       color="forestgreen", legend="Injured")

# p3.xaxis.axis_label = 'Region'
p4.yaxis.axis_label = 'Percentage'

p4.x_range.range_padding = 0.1
p4.xgrid.grid_line_color = None
p4.legend.location = "top_left"
p4.legend.orientation = "vertical"
p4.xaxis.major_label_orientation = math.pi/4
p4.axis.major_label_text_font_size = "7pt"
p4.legend.label_text_font_size = "9pt"

hover = HoverTool(tooltips = [('Terrorist Incidents', '@MN_Attacks_Percent%'),
                             ('Casualties', '@MN_Fatalities_Percent'),
                             ('Injured', '@MN_Wounded_Percent')])
p4.add_tools(hover)
show(p4)

### Exploring South Asia

In [ ]:
South_Asia = GTD[GTD['region_txt'] == 'South Asia']
South_Asia.shape

In [ ]:
len(South_Asia['country_txt'].unique())

In [ ]:
SA_country = South_Asia.groupby(['country_txt']).agg({'eventid' :'count', 'nkill' : 'sum', 'nwound':'sum'}).reset_index()
SA_country= SA_country.rename(columns = {'country_txt' : 'Country',
                                        'eventid' : 'Attacks',
                                        'nkill' : 'Fatalities',
                                        'nwound' : 'Wounded'})
SA_country['SA_Attacks_Percent'] = SA_country['Attacks']*100/SA_country['Attacks'].sum()
SA_country['SA_Fatalities_Percent'] = SA_country['Fatalities']*100/SA_country['Fatalities'].sum()
SA_country['SA_Wounded_Percent'] = SA_country['Wounded']*100/SA_country['Wounded'].sum()

SA_country

In [ ]:
source_SA = ColumnDataSource(data=SA_country)
SA_Country = source_SA.data['Country'].tolist()

p5 = figure(x_range=SA_Country, y_range=(0, 45), plot_height=350, title="Terrorist events, Casualties and Injuries in South Asia")

output_file("/Users/poonamarora/Desktop/Aleph/Pictures/html/South_Asia.html")

p5.vbar(x=dodge('Country', -0.25, range=p5.x_range), top='SA_Attacks_Percent', width=0.2, source=source_SA,
       color="darkturquoise", legend="Total Incidents")

p5.vbar(x=dodge('Country',  0.01,  range=p5.x_range), top='SA_Fatalities_Percent', width=0.2, source=source_SA,
       color="indianred", legend="Casualities")

p5.vbar(x=dodge('Country',  0.27,  range=p5.x_range), top='SA_Wounded_Percent', width=0.2, source=source_SA,
       color="forestgreen", legend="Injured")

p5.x_range.range_padding = 0.1
p5.xgrid.grid_line_color = None
p5.legend.location = "top_right"
p5.legend.orientation = "vertical"
p5.xaxis.major_label_orientation = math.pi/4
p5.axis.major_label_text_font_size = "7pt"
p5.legend.label_text_font_size = "9pt"
p5.yaxis.axis_label = 'Percentage'

hover = HoverTool(tooltips = [('Terrorist Events', '@SA_Attacks_Percent%'),
                             ('Casualties', '@SA_Fatalities_Percent%'),
                             ('Injured', '@SA_Wounded_Percent%')])
p5.add_tools(hover)
show(p5)

### Exploring sub saharan africa

In [ ]:
SubSaharan_Africa = GTD[GTD['region_txt'] == 'Sub-Saharan Africa']
SubSaharan_Africa.shape

In [ ]:
len(SubSaharan_Africa['country_txt'].unique())

In [ ]:
SSA_country = SubSaharan_Africa.groupby('country_txt').agg({'eventid':'count', 'nkill': 'sum', 'nwound':'sum'}).reset_index()
SSA_country= SSA_country.rename(columns = {'country_txt' : 'Country',
                                        'eventid' : 'Attacks',
                                        'nkill' : 'Fatalities',
                                          'nwound' : 'Wounded'})
SSA_country['SSA_Attacks_Percent'] = SSA_country['Attacks']*100/SSA_country['Attacks'].sum()
SSA_country['SSA_Fatalities_Percent'] = SSA_country['Fatalities']*100/SSA_country['Fatalities'].sum()
SSA_country['SSA_Wounded_Percent'] = SSA_country['Wounded']*100/SSA_country['Wounded'].sum()

SSA_country

In [ ]:
SSA_country = SSA_country[SSA_country['SSA_Attacks_Percent'] >= 3.0]

In [ ]:
SSA_country

In [ ]:
source_SSA = ColumnDataSource(data=SSA_country)
SSA_country = source_SSA.data['Country'].tolist()

p6 = figure(x_range=SSA_country, y_range=(0, 35), plot_height=350, title="Terrorist events, Casualties and Injuries in Sub Saharan Africa")

output_file("/Users/poonamarora/Desktop/Aleph/Pictures/html/Sub_Saharan_Africa.html")

p6.vbar(x=dodge('Country', -0.25, range=p6.x_range), top='SSA_Attacks_Percent', width=0.2, source=source_SSA,
       color="darkturquoise", legend="Total Incidents")

p6.vbar(x=dodge('Country',  0.01,  range=p6.x_range), top='SSA_Fatalities_Percent', width=0.2, source=source_SSA,
       color="indianred", legend="Casualties")

p6.vbar(x=dodge('Country',  0.27,  range=p6.x_range), top='SSA_Wounded_Percent', width=0.2, source=source_SSA,
       color="forestgreen", legend="Injured")


p6.x_range.range_padding = 0.1
p6.xgrid.grid_line_color = None
p6.legend.location = "top_right"
p6.legend.orientation = "vertical"
p6.xaxis.major_label_orientation = math.pi/4
p6.axis.major_label_text_font_size = "7pt"
p6.legend.label_text_font_size = "9pt"
p6.yaxis.axis_label = 'Percentage'
hover = HoverTool(tooltips = [('Terrorist Events', '@SSA_Attacks_Percent%'),
                             ('Casualties', '@SSA_Fatalities_Percent%'),
                             ('Injured', '@SSA_Wounded_Percent%')])
p6.add_tools(hover)
show(p6)

### Attack type distribution - Global and UK

In [23]:
ATT = UK.groupby('attacktype1_txt').agg({'eventid':'count', 'nkill' :'sum', 'nwound' : 'sum'}).reset_index()
ATT = ATT.rename(columns= {'attacktype1_txt' : 'AttackType',
                          'eventid' :'Attacks',
                          'nkill' : 'Fatalities',
                          'nwound' : 'Wounded'})
ATT['Attacks_Percent'] = ATT['Attacks']*100/ATT['Attacks'].sum()
ATT['angle'] = ATT['Attacks_Percent']/ATT['Attacks_Percent'].sum() * 2*pi
x=ATT['AttackType'].value_counts()
ATT['color'] = Category20c[len(x)]
ATT['Fatalities_Percent'] = ATT['Fatalities']*100/ATT['Fatalities'].sum()
ATT['Wounded_Percent'] = ATT['Wounded']*100/ATT['Wounded'].sum()


ATT

,AttackType,Attacks,Fatalities,Wounded,Attacks_Percent,angle,color,Fatalities_Percent,Wounded_Percent
0,Armed Assault,707,424.000,488.000,13.252,0.833,#3182bd,12.430,7.950
1,Assassination,1653,1587.000,269.000,30.984,1.947,#6baed6,46.526,4.383
2,Bombing/Explosion,2115,1309.000,5214.000,39.644,2.491,#9ecae1,38.376,84.946
3,Facility/Infrastructure Attack,683,10.000,60.000,12.802,0.804,#c6dbef,0.293,0.978
4,Hijacking,20,2.000,1.000,0.375,0.024,#e6550d,0.059,0.016
5,Hostage Taking (Barricade Incident),14,9.000,7.000,0.262,0.016,#fd8d3c,0.264,0.114
6,Hostage Taking (Kidnapping),53,38.000,11.000,0.993,0.062,#fdae6b,1.114,0.179
7,Unarmed Assault,50,15.000,52.000,0.937,0.059,#fdd0a2,0.440,0.847
8,Unknown,40,17.000,36.000,0.750,0.047,#31a354,0.498,0.587


In [ ]:
output_file("/Users/poonamarora/Desktop/Aleph/Pictures/html/AttackTypes.html")

p7 = figure(plot_height=550, toolbar_location=None,
           tools="hover", tooltips="@AttackType: @Attacks_Percent{0.2f} %", x_range=(-.5, .5),
           title = "Types of Attacks globally over the uear(1970-2018)")

p7.annular_wedge(x=0, y=1,  inner_radius=0.15, outer_radius=0.25, direction="anticlock",
                start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='AttackType',source=ATT)

p7.legend.label_text_font_size = "4.4pt"
p7.axis.axis_label=None
p7.axis.visible=False
p7.grid.grid_line_color = None
p7.legend.location = "top_left"
p7.legend.orientation = "vertical"
p7.legend.click_policy='mute'
show(p7) 

In [ ]:
ATT = UK.groupby(['attacktype1_txt', 'iyear']).agg({'eventid':'count', 'nkill' :'sum', 'nwound' : 'sum'}).reset_index()
ATT = ATT.rename(columns= {'iyear':'Year',
                           'attacktype1_txt' : 'AttackType',
                          'eventid' :'Attacks',
                          'nkill' : 'Fatalities',
                          'nwound' : 'Wounded'})
ATT['Attacks_Percent'] = ATT['Attacks']*100/ATT['Attacks'].sum()
ATT['angle'] = ATT['Attacks_Percent']/ATT['Attacks_Percent'].sum() * 2*pi
x=ATT['AttackType'].value_counts()
ATT['color'] = Category20c[len(x)]
ATT['Fatalities_Percent'] = ATT['Fatalities']*100/ATT['Fatalities'].sum()
ATT['Wounded_Percent'] = ATT['Wounded']*100/ATT['Wounded'].sum()


ATT

### Target type distribution

In [ ]:
TT = UK.groupby('targtype1_txt').agg({'eventid':'count', 'nkill' :'sum', 'nwound' : 'sum'}).reset_index()
TT = TT.rename(columns= {'targtype1_txt' : 'TargetType',
                        'eventid' :'Attacks',
                          'nkill' : 'Fatalities',
                          'nwound' : 'Wounded'})
TT['Attacks_Percent'] = TT['Attacks']*100/TT['Attacks'].sum()
TT = TT[TT.Attacks_Percent > 2.0]
TT['angle'] = TT['Attacks_Percent']/TT['Attacks_Percent'].sum() * 2*pi
x=TT['TargetType'].value_counts()
TT['color'] = Category20c[len(x)]
TT['Fatalities_Percent'] = TT['Fatalities']*100/TT['Fatalities'].sum()
TT['Wounded_Percent'] = TT['Wounded']*100/TT['Wounded'].sum()
TT

In [ ]:
output_file("/Users/poonamarora/Desktop/Aleph/Pictures/html/TargetTypes.html")

p8 = figure(plot_height=550, toolbar_location=None,
           tools="hover", tooltips="@TargetType: @Attacks_Percent{0.2f} %", x_range=(-.5, .5),
           title = "Types of Targets globally over the uear(1970-2018)")

p8.annular_wedge(x=0, y=1,  inner_radius=0.15, outer_radius=0.25, direction="anticlock",
                start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='TargetType',source=TT, alpha =0.8)

p8.legend.label_text_font_size = "5.3pt"
p8.axis.axis_label=None
p8.axis.visible=False
p8.grid.grid_line_color = None
p8.legend.location = "top_left"
p8.legend.orientation = "vertical"
p8.legend.click_policy='mute'
show(p8) 

### Weapon type distribution

In [ ]:
WT = GTD.groupby('weaptype1_txt').agg({'eventid':'count', 'nkill' :'sum', 'nwound' : 'sum'}).reset_index()
WT = WT.rename(columns= {'weaptype1_txt' : 'WeaponType',
                        'eventid' :'Attacks',
                         'nkill' : 'Fatalities',
                          'nwound' : 'Wounded'})
WT['Attacks_Percent'] = WT['Attacks']*100/WT['Attacks'].sum()
WT = WT[WT.Attacks_Percent > 2.0]
WT['angle'] = WT['Attacks_Percent']/WT['Attacks_Percent'].sum() * 2*pi
x=WT['WeaponType'].value_counts()
WT['color'] = Category10[len(x)]
WT['Fatalities_Percent'] = WT['Fatalities']*100/WT['Fatalities'].sum()
WT['Wounded_Percent'] = WT['Wounded']*100/WT['Wounded'].sum()
WT

In [ ]:
output_file("/Users/poonamarora/Desktop/Aleph/Pictures/html/WeaponTypes.html")

p9 = figure(plot_height=550, toolbar_location=None,
           tools="hover", tooltips="@WeaponType: @Attacks_Percent{0.2f} %", x_range=(-.5, .5),
           title = "Types of Weapons used globally over the uear(1970-2018)")

p9.annular_wedge(x=0, y=1,  inner_radius=0.15, outer_radius=0.25, direction="anticlock",
                start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='WeaponType',source=WT, alpha =0.6)

p9.legend.label_text_font_size = "5.3pt"
p9.axis.axis_label=None
p9.axis.visible=False
p9.grid.grid_line_color = None
p9.legend.location = "top_left"
p9.legend.orientation = "vertical"
p9.legend.click_policy='mute'
show(p9) 

### Property Damage

In [16]:
prop = GTD_full[GTD_full.property == 1]
prop.shape

(96750, 135)

In [17]:
prop.shape[0] /GTD_full.shape[0]

0.505316926419588

In [18]:
pd.set_option('display.max_colwidth', -1)
t = prop[prop.propextent == 1] 
t[['eventid', 'iyear','imonth','country_txt','scite1', 'scite2','scite3']]

,eventid,iyear,imonth,country_txt,scite1,scite2,scite3
50994,199204100007,1992,4,United Kingdom,NaN,NaN,NaN
62649,199606150004,1996,6,United Kingdom,"""1996: Huge explosion rocks central Manchester,"" BBC News, On This Day 1959-2005, June 15.","James Cusick, ""The Manchester Bombing: Device was as big as anything to hit mainland Britain,"" The Independent, June 17, 1996.","""Manchester bomb: no justice,"" BBC, June 15, 2006."
73137,200109110004,2001,9,United States,"United States Government, The 9/11 Commission Report: Final Report of the National Commission on Terrorist Attacks Upon the United States. Washington, DC: Government Printing Office, 2004.","Lindsay Kines, “United States on high alert after thousands feared dead: Airlines initially blame hijackers for attacks on the Pentagon in Washington, New York's World Trade Center,” The Vancouver Sun, September 11, 2001.","Joe Frolick, “Hijackers Ram Two Airliners Into World Trade Center, Third Plane Slams Into Pentagon, Fourth Crashes Near Pittsburgh; Thousands Die; Bush Vows Swift Revenge,” The Plain Dealer, September 12, 2001."
73138,200109110005,2001,9,United States,"United States Government, The 9/11 Commission Report: Final Report of the National Commission on Terrorist Attacks Upon the United States. Washington, DC: Government Printing Office, 2004.","Lindsay Kines, “United States on high alert after thousands feared dead: Airlines initially blame hijackers for attacks on the Pentagon in Washington, New York's World Trade Center,” The Vancouver Sun, September 11, 2001.","Joe Frolick, “Hijackers Ram Two Airliners Into World Trade Center, Third Plane Slams Into Pentagon, Fourth Crashes Near Pittsburgh; Thousands Die; Bush Vows Swift Revenge,” The Plain Dealer, September 12, 2001."
73139,200109110006,2001,9,United States,"United States Government, The 9/11 Commission Report: Final Report of the National Commission on Terrorist Attacks Upon the United States. Washington, DC: Government Printing Office, 2004.","Lindsay Kines, “United States on high alert after thousands feared dead: Airlines initially blame hijackers for attacks on the Pentagon in Washington, New York's World Trade Center,” The Vancouver Sun, September 11, 2001.","""Plane Crashes Into Pentagon; Troops Deployed in Response,"" Associated Press, September 11, 2001."
73140,200109110007,2001,9,United States,"United States Government, The 9/11 Commission Report: Final Report of the National Commission on Terrorist Attacks Upon the United States. Washington, DC: Government Printing Office, 2004.","Lindsay Kines, “United States on high alert after thousands feared dead: Airlines initially blame hijackers for attacks on the Pentagon in Washington, New York's World Trade Center,” The Vancouver Sun, September 11, 2001.","“Plane Crashes Into Pentagon; Troops Deployed in Response,” Associated Press, September 11, 2001."


In [19]:
def calc(row):
    if row['PropertDamage'] == 1.0:
        return ' Catastrophic (likely ≥ $1 billion)'
    elif row['PropertDamage'] == 2.0:
        return ' Major (likely ≥ $1 million but < $1 billion)'
    elif row['PropertDamage'] == 3.0:
        return 'Minor (likely < $1 million)'
    else: 
        return 'Unknown'
    
prop = prop.groupby(['propextent']).agg({'eventid':'count', 'propvalue' : 'sum'}).reset_index()
prop = prop.rename(columns={'propextent' : 'PropertDamage',
                            'eventid' :'Attacks',
                           'propvalue': 'PropertyValue'})
# prop['PropertDamage'].rename_categories({'1.0': 'Catastrophic', '2.0': 'Major',  '3.0': 'Minor' ,'4.0': 'Unknown'})
prop['Attacks_Percent'] = prop['Attacks']*100/prop['Attacks'].sum()
prop['category'] = prop.apply(lambda row: calc(row), axis=1)
prop['angle'] = prop['Attacks_Percent']/prop['Attacks_Percent'].sum() * 2*pi
x=prop['category'].value_counts()
prop['color'] = Category10[len(x)]

In [20]:
prop

,PropertDamage,Attacks,PropertyValue,Attacks_Percent,category,angle,color
0,1.000,6,3779120000.000,0.009,Catastrophic (likely ≥ $1 billion),0.001,#1f77b4
1,2.000,915,3866989276.840,1.432,Major (likely ≥ $1 million but < $1 billion),0.090,#ff7f0e
2,3.000,45593,503265607.710,71.371,Minor (likely < $1 million),4.484,#2ca02c
3,4.000,17368,6654755.000,27.188,Unknown,1.708,#d62728


In [ ]:
output_file("/Users/poonamarora/Desktop/Aleph/Pictures/html/PropertyDamage.html")

p10 = figure(plot_height=550, toolbar_location=None,
           tools="hover", tooltips="@category: @Attacks_Percent{0.2f} %", x_range=(-.5, .5),
           title = "Property damage due to Terrorist activities(1970-2018)")

# p10.annular_wedge(x=0, y=1,  inner_radius=0.15, outer_radius=0.25, direction="anticlock",
#                 start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
#         line_color="white", fill_color='color', legend='category',source=prop, alpha = 0.7)

p10.wedge(x=0, y=1, radius=0.30, direction="anticlock",
                start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='category',source=prop, alpha = 0.6)


p10.legend.label_text_font_size = "5pt"
p10.axis.axis_label=None
p10.axis.visible=False
p10.grid.grid_line_color = None
p10.legend.location = "top_left"
p10.legend.orientation = "vertical"
p10.legend.click_policy='mute'
show(p10) 

### Hostage events over the world

In [ ]:
hstg = GTD[GTD.ishostkid == 1]
hstg.shape

In [ ]:
hstg.shape[0]/GTD.shape[0]

### Citywise terror attacks

In [ ]:
Country = GTD_full.groupby(['country_txt']).agg({'eventid' : 'count', 'nkill' :'sum', 'nwound' :'sum'}).reset_index()
Country = Country.rename(columns = {'country_txt' : 'Country',
                                    'eventid' : 'Attacks'})
Country['CN_Attacks_Percent'] = Country['Attacks']*100/Country['Attacks'].sum()
Country = Country[Country.CN_Attacks_Percent >= 2.0]
Country

In [ ]:
Troubled_countries = Country.Country.tolist()
Troubled_countries

In [ ]:
City = GTD_full[['eventid', 'city','country_txt','iyear','attacktype1_txt','targtype1_txt', 'nkill']]
City = City[(City.country_txt.isin(Troubled_countries)) & (City.attacktype1_txt != 'Unknown')]
City.shape

In [ ]:
City.columns

In [ ]:
City.head()

In [ ]:
City_CN = City.groupby(['country_txt', 'attacktype1_txt', 'iyear']).agg({'eventid' : 'count', 'nkill' :'sum'}).reset_index()
City_CN = City_CN.rename(columns = {'country_txt' : 'Country',
                             'eventid' : 'Attacks',
                             'nkill' : 'Fatalities'})
# City_CN['CN_Attacks_Percent'] = City_CN['Attacks']*100/City_CN['Attacks'].sum()
# City_CN['Cn_Fatalities_Percent'] = City_CN['Fatalities']*100/City_CN['Fatalities'].sum()
City_CN['Attacks'] = np.log(City_CN['Attacks'])
City_CN['Fatalities'] = np.log(City_CN['Fatalities'])
City_CN

In [ ]:
### HeatMap
source = ColumnDataSource(City_CN)

# this is the colormap from the original NYTimes plot
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=City_CN.Attacks.min(), high=City_CN.Attacks.max())

p12 = figure(plot_width=800, plot_height=300, title="Terrorist Activities in most troubled countries",
           x_range=list(City_CN.Country.unique()), y_range=list(City_CN.attacktype1_txt.unique()),
           toolbar_location=None, tools="", x_axis_location="above")

p12.rect(x="Country", y="attacktype1_txt", width=1, height=1, source=source,
       line_color=None, fill_color=transform('Attacks', mapper))

color_bar = ColorBar(color_mapper=mapper, location=(0, 0),
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%.1f"))

p12.add_layout(color_bar, 'right')

p12.axis.axis_line_color = None
p12.axis.major_tick_line_color = None
p12.axis.major_label_text_font_size = "7pt"
p12.axis.major_label_standoff = 0
p12.xaxis.major_label_orientation = 1.0
hover = HoverTool(tooltips = [
    ('Year', '@iyear'),
    ('Country', '@Country'),
    ('Attack Type', '@attacktype1_txt'),
    ('Global Events', '@Attacks')])

p12.add_tools(hover)
def update_plot(attr, old, new):
    yr = slider.value
    new_data = City_CN['iyear']
    p.title.text = 'Terrorist Activities, %d' %yr
    
    
slider = Slider(title = 'Year',start = 1970, end = 2018, step = 1, value = 2001)
slider.on_change('value', update_plot)

layout = column(p12,widgetbox(slider))
curdoc().add_root(layout)
# #Display plot inline in Jupyter notebook
output_notebook()
# #Display plot
show(layout)

In [ ]:
GTD_full = pd.read_pickle("/Users/poonamarora/Desktop/Aleph/Datasets/GTD_Dataset.pkl")

Country = GTD_full.groupby(['country_txt']).agg({'eventid' : 'count', 'nkill' :'sum', 'nwound' :'sum'}).reset_index()
Country = Country.rename(columns = {'country_txt' : 'Country',
                                    'eventid' : 'Attacks'})
Country['CN_Attacks_Percent'] = Country['Attacks']*100/Country['Attacks'].sum()
Country = Country[Country.CN_Attacks_Percent >= 2.0]

# Country
Troubled_countries = Country.Country.tolist()

City = GTD_full[['eventid', 'city','country_txt','iyear','attacktype1_txt','targtype1_txt']]
City = City[(City.country_txt.isin(Troubled_countries)) & (City.attacktype1_txt != 'Unknown')]
# City.shape

City_CN = City.groupby(['country_txt', 'attacktype1_txt', 'iyear']).agg({'eventid' : 'count'}).reset_index()
City_CN = City_CN.rename(columns = {'country_txt' : 'Country',
                             'eventid' : 'Attacks'})
City_CN['Attacks'] = np.log(City_CN['Attacks'])
# City_CN['Fatalities'] = np.log(City_CN['Fatalities'])


In [ ]:
City_CN

In [ ]:
def data(selectedYear):
    yr= selectedYear
    city_data = City_CN[City_CN.iyear == yr]
    return(city_data)

In [ ]:
data(1993)

In [ ]:
GTD[GTD.iyear == 1993]

In [ ]:
GTD.attacktype1_txt.unique()

In [ ]:
# from bokeh.io import show, output_file
# from bokeh.models import ColumnDataSource, FactorRange
# from bokeh.plotting import figure
# from bokeh.transform import factor_cmap

# countries = list(City_CN.Country.unique())
# attacktype = list(City_CN.attacktype1_txt.unique())

# palette = ["#c9d9d3", "#718dbf", "#e84d60","#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878"]

# x = [(a, b) for a in countries for b in attacktype ]
# counts = list(City_CN.Cn_Fatalities_Percent) # like an hstack

# source = ColumnDataSource(data=dict(x=x, counts=counts))

# p = figure(x_range=FactorRange(*x), plot_height=350, title="Terror",
#            toolbar_location=None, tools="")

# p.vbar(x='x', top='counts', width=1, source=source, line_color="white",
#        fill_color=factor_cmap('x', palette=palette, factors=attacktype, start=1, end=7))

# p.y_range.start = 0
# p.x_range.range_padding = 0.1
# p.xaxis.major_label_orientation = 1
# p.xgrid.grid_line_color = None

# show(p)

In [ ]:
GTD.attacktype2_txt.unique()

In [ ]:
GTD.attacktype3_txt.unique()

In [ ]:
GTD.targtype1_txt.unique()

In [ ]:
GTD.targtype2_txt.unique()

In [ ]:
GTD.city.unique()

In [ ]:
GTD.weaptype1_txt.unique()

In [ ]:
# make a grid
grid = gridplot([[p1, p2], [p3, p4]], plot_width=500, plot_height=500)
show(grid)